<a href="https://colab.research.google.com/github/Ian326/TI3001C/blob/main/E5_Construcci%C3%B3n_y_transformaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recolección y descripción de datos
---
***Equipo:***

**Ruth Jiménez Vázquez - A01351923**

**José Ignacio Gómez Moreno - A01067362**

**Harry Hernández Grande - A01736341**

**Ian Joab Padrón Corona - A01708940**

---



## Carga de datos

In [ ]:
'''
===============================================================================
Librerias de Python a utilizar en el proyecto
===============================================================================
pandas: Libreria de manipulacion de datos en DataFrames
numpy: Libreria de manipulacion de datos en arrays o funciones matematicas
missingno: Libreria de visualizacion de datos nulos
CountEncoder: Convertir variables categoricas en numericas asignandoles un valor de acuerdo a su relevancia
OneHotEncoder: Convertir variables categoricas en numericas creando columnas binarias
matplotlib: Libreria de visualizacion de datos a traves de graficos
seaborn: Libreria de visualizacion de datos a traves de graficos. (Extension de matplotlib)
PowerTransformer: Transformar variables numericas a traves de una transformacion de potencia. Para normalizar los datos
stats: Libreria de estadisticas para realizar pruebas de normalidad
accuracy_score: Funcion para calcular la precision de un modelo de clasificacion
'''

import pandas as pd
import numpy as np

import missingno as msno

from category_encoders import CountEncoder
from category_encoders import OneHotEncoder

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import PowerTransformer

import scipy.stats as stats

from sklearn.metrics import accuracy_score

In [ ]:
print(pd.read_csv('../content/vista_repdetail_tec.csv', low_memory=False).info())
# Como se puede observar, el archivo .CSV no contiene el nombre de las columnas a utilizar.
# Debido a esto, utiliza el primer registro como nombre de las columnas.
# Por tanto, se debera asignar manualmente el nombre de las columnas según el diccionario de datos 
# proporcionado por el cliente.

In [ ]:
# Nombres de las columnas según el Diccionario de Datos proporcionado
columns = [
    "Column", "Taller", "OrderID", "kMS", "OpenedDate", "ClosedDate", "Status",
    "lagdias", "laghoras", "ClosedDay", "ClosedMonth", "ClosedYear", "OpenedDay",
    "OpenedMonth", "OpenedYear", "CreatedBy", "ClosedBy", "RepReason",
    "OrderType", "Supplier", "COMPCDKEY", "COMPCODE", "DESCRIP", "QTYRCVD",
    "QTYREQD", "CHGAMT", "TAXAMT", "SUBTOTAL", "TOTAL", "UnitID", "UnitType",
    "UnitYear", "EC", "Flota", "ReasonType", "COMPLAINT", "Jobcode", "TipoLinea",
    "NumParte", "FechaPromesa", "Estimate", "CompleteDay", "CompleteMonth",
    "CompleteYear", "CompleteDate", "MECHANIC", "CITA"
]

# Cargar el archivo .CSV con los nombres de las columnas asignados a un DataFrame 'data'
data = pd.read_csv('../content/vista_repdetail_tec.csv', names=columns, low_memory=False)
# Mostrar la informacion del DataFrame creado
data.info()

In [ ]:
# Visualizacion de una muestra de los datos en el DataFrame
data.sample(3)

## Documentación de los esquemas de datos

* ¿Cuál es nuestra fuente de datos?

Actualmente, la información es recabada por nuestro SocioFormador TDR. Tenemos acceso a un escritorio remoto que está a su vez conectado con un servidor de Microsoft SQL Server.

A partir de dicho servidor, llamamos a una de las vistas a las que nos dieron acceso, para recabar toda la información sobre las reparaciones de su flota.


* ¿Cómo obtenemos la información?

A traves de una Query de SQL a la que nos autorizaron, que seria la siguiente:


```
SELECT * FROM [TMWSuite].[dbo].[vista_repdetail_tec]
```




## Diccionario de datos

| Column        | Descripción                                                                                       |
| ------------- | ------------------------------------------------------------------------------------------------- |
| OrderID       | ID de la orden de reparación                                                                      |
| Taller        | Acrónimo del taller donde se realizó la reparación                                                |
| kMS         | Kilometros viajados por la unidad                                                                     |
| OpenedDate    | Fecha de apertura de la orden de reparación                                                       |
| ClosedDate    | Fecha de cierre de la orden de reparación                                                         |
| Status        | Estatus de la orden de reparación                                                                 |
| lagdias       | Días entre cierre y apertura de la orden de reparación                                            |
| laghoras      | Horas entre cierre y apertura de la orden de reparación                                           |
| ClosedDay     | Día del mes en que se cerró la orden de reparación                                                |
| ClosedMonth   | Mes que se cerró la orden de reparación                                                           |
| ClosedYear    | Año en que se cerró la orden de reparación                                                        |
| OpenedDay     | Día del mes en que se abrió la orden de reparación                                                |
| OpenedMonth   | Mes que se abrió la orden de reparación                                                           |
| OpenedYear    | Año en que se abrió la orden de reparación                                                        |
| CreatedBy     | Usuario que creó la orden de reparación                                                           |
| ClosedBy      | Usuario que cerro la orden de reparación                                                          |
| RepReason     | Razón de reparación                                                                               |
| OrderType     | Tipo de orden. Standar es en uno de nuestros talleres propios. Vendor es con un proveedor externo |
| Supplier      | Proveedor que realizó la reparación                                                               |
| COMPCDKEY     | Codigo de componente (No aporta info importante)                                                  |
| COMPCODE      | Codigo utilizado para identificar el tipo de reparación                                           |
| DESCRIP       | Descripción de la reparación                                                                      |
| QTYRCVD       | Cantidad de piezas o trabajo recibidas                                                            |
| QTYREQD       | Cantidad de piezas o trabajo requeridas                                                           |
| CHGAMT        | Costo unitario                                                                                    |
| TAXAMT        | IVA                                                                                               |
| SUBTOTAL      | Subtotal de la reparación (No incluye IVA)                                                        |
| TOTAL         | Total de la reparación (Incluye IVA)                                                              |
| UnitID        | Numero de unidad                                                                                  |
| UnitType      | Tipo de unidad (Tracto, Caja Seca, Dolly, etc.)                                                   |
| UnitYear      | Año del modelo de la unidad                                                                       |
| EC            | Equipo administrativo que se encarga de la operación de un conjunto de proyectos                  |
| Flota         | Nombre del proyecto en que trabaja el tracto. (Cliente)                                           |
| ReasonType    | Razón (No aporta info importante)                                                                 |
| COMPLAINT     | Queja reportada                                                                                   |
| Jobcode       | Numero para identificar el tipo de reparación                                                     |
| TipoLinea     | Describe el tipo de línea en la orden de reparación (Piezas, mano de obra, servicios, etc.)       |
| NumParte      | Numero de parte descarga en la línea de reparación                                                |
| FechaPromesa  | Fecha de promesa para terminar las reparaciones de la unidad                                      |
| Estimate      | Monto estimado (No lo utilizamos)                                                                 |
| CompleteDay   | Día del mes en que se completó la orden                                                           |
| CompleteMonth | Mes que se completó la orden de reparación                                                        |
| CompleteYear  | Año en que se completó la orden de reparación                                                     |
| CompleteDate  | Fecha en que se completó la orden de reparación                                                   |
| MECHANIC      | Mecánico interno que realizó la reparación                                                        |
| CITA          | Indica si la unidad tenía una cita agenda previamente                                             |

## Análisis de la estructura de los datos

* ¿Que tipo de datos tenemos? **Tenemos 13 de tipo float, 4 int, y 30 object.**
* ¿Requerimos modificar el tipo de datos de alguna columna? **Sí, para ser precisos, debemos convertir las siguientes columnas:**
  * **A int64**:
    * lagdias: Son cantidad de dias, no pueden venir con decimales
    * laghoras: Son cantidad de horas, no pueden venir con decimales
    * ClosedDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
    * QTYRCVD: Es una cantidad de piezas, no puede venir con decimales
    * QTYREQD: Es una cantidad de piezas, no puede venir con decimales
    * CompleteDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
    * CompleteYear: Es un año (no puede tener punto decimal o ser un string)
  
  * **A datetime64**
    * OpenedDate: Actualmente es un object, debe ser fecha
    * ClosedDate: Actualmente es un object, debe ser fecha
    * FechaPromesa: Actualmente es un object, debe ser fecha
    * CompleteDate: Actualmente es un object, debe ser fecha

* ¿Existen nulos? **Sí, actualmente hay 689,643 registros faltantes en toda la informacion que tenemos**

In [ ]:
# Conteo de valores nulos en el DataFrame
data.isnull().sum()

In [ ]:
print(data.isnull().sum().sum()) # <- Cantidad de nulos

## Perfiles involucrados

*   Equipo de TDR que nos puede ayudar con los datos (Identificar por nombre y cargo)


### Ing. Emmanuel Vargas Pérez

**Jefe de Administración de Equipo**

Nos proporcionará información sobre el proyecto que desarrollaremos (Sayer), incluyendo configuraciones de las flotas de tractocamiones y las fallas más comunes que suelen presentar estos vehículos.






### Ing. Mauricio Romo Ávila

**Subdirector de Mantenimiento**

Nos brindará información sobre el mantenimiento de los tractocamiones, incluyendo las fallas comunes, los costos asociados a estas fallas, las ubicaciones donde ocurren con mayor frecuencia, el número de talleres disponibles (tanto propios como generales), y responderá dudas sobre sus mantenimientos correctivos.


### Ing. Brandon Velazco

**Jefe de Taller**

Aportará información relevante sobre los tractocamiones, incluyendo detalles sobre las partes y componentes específicos que suelen fallar, en qué consisten estas fallas, así como la duración y el costo de las mismas.

*   Los profesores nos brindarán las herramientas y conocimientos
necesarios para lograr las metas y objetivos.

## Limpieza de Indices y Columnas

* ¿Qué indices vamos a utilizar?
 **La mejor opcion seria dejar un valor numérico discreto como índice, con el fin de que no tenga valores repetidos**

* ¿Qué columnas requieren un cambio de Tipo de Dato?
  * **A int64**:
      * lagdias: Son cantidad de días (enteros), no pueden venir con decimales
      * laghoras: Son cantidad de horas (enteros), no pueden venir con decimales
      * ClosedDay: Es el número del día en que fue cerrada (puede ser de 1 - 31)
      * QTYRCVD: Es una cantidad de piezas (enteras), no puede venir con decimales
      * QTYREQD: Es una cantidad de piezas (enteras), no puede venir con decimales
      * CompleteDay: Es el número del día en que fue cerrada (puede ser de 1 - 31)
      * CompleteYear: Es un año (no puede tener punto decimal o ser un string)

  * **A float64**:
    * Estimate: Se encuentra como object actualmente, debe ser un precio en decimal
  
  * **A datetime64**
    * OpenedDate: Actualmente es un object, debe ser fecha
    * ClosedDate: Actualmente es un object, debe ser fecha
    * FechaPromesa: Actualmente es un object, debe ser fecha
    * CompleteDate: Actualmente es un object, debe ser fecha


In [ ]:
# Convertir las columnas requeridas a 'DateTime'
data['OpenedDate'] = pd.to_datetime(data['OpenedDate'])
data['ClosedDate'] = pd.to_datetime(data['ClosedDate'])
data['FechaPromesa'] = pd.to_datetime(data['FechaPromesa'])
data['CompleteDate'] = pd.to_datetime(data['CompleteDate'])

In [ ]:
# Redondear los decimales de las columnas requeridas a 0, para posteriormente convertirlas a 'Int'
data['lagdias'] = data['lagdias'].round(0)
data['laghoras'] = data['laghoras'].round(0)
data['ClosedDay'] = data['ClosedDay'].round(0)
data['ClosedYear'] = data['ClosedYear'].round(0)
data['QTYRCVD'] = data['QTYRCVD'].round(0)
data['QTYREQD'] = data['QTYREQD'].round(0)
data['CompleteDay'] = data['CompleteDay'].round(0)
data['CompleteYear'] = data['CompleteYear'].round(0)

In [ ]:
# Llenar los valores nulos de las columnas requeridas con 0, para posteriormente convertirlas a 'Int'
data['lagdias'] = data['lagdias'].fillna(0)
data['laghoras'] = data['laghoras'].fillna(0)
data['ClosedDay'] = data['ClosedDay'].fillna(0)
data['ClosedYear'] = data['ClosedYear'].fillna(0)
data['QTYRCVD'] = data['QTYRCVD'].fillna(0)
data['QTYREQD'] = data['QTYREQD'].fillna(0)
data['CompleteDay'] = data['CompleteDay'].fillna(0)
data['CompleteYear'] = data['CompleteYear'].fillna(0)

In [ ]:
# Convertir las columnas requeridas a 'Int'
data['lagdias'] = data['lagdias'].astype(int)
data['laghoras'] = data['laghoras'].astype(int)
data['ClosedDay'] = data['ClosedDay'].astype(int)
data['ClosedYear'] = data['ClosedYear'].astype(int)
data['QTYRCVD'] = data['QTYRCVD'].astype(int)
data['QTYREQD'] = data['QTYREQD'].astype(int)
data['CompleteDay'] = data['CompleteDay'].astype(int)
data['CompleteYear'] = data['CompleteYear'].astype(int)

In [ ]:
# Convertir columnas requeridas a 'Float'
# data['Estimate'] = data['Estimate'].astype(float)
# No se puede, porque la informacion en la columna 'Estimate' no es realmente un estimado

In [ ]:
# Volver a revisar el tipo de dato de las columnas
data.info()

## Análisis de los datos (Descriptiores Estadisticos)

In [ ]:
# Seleccionar sólo las variables relevantes para las estadisticas
data_descriptive = data[['lagdias', 'laghoras','TOTAL','UnitYear',
                      'RepReason', 'UnitType', 'TipoLinea', 'CITA', 'COMPLAINT']]

data_descriptive.sample(10)

**Estadística descriptiva de las variables numéricas**

In [ ]:
# Estadística descriptiva de las variables numéricas
data_descriptive.describe()

| Variable numérica | Descripción | Rango | Desviación estándar |
|-----------|-----------|-----------|-----------|
| lagdias    | Días entre cierre y apertura de la orden de reparación   | 655    | Datos no tan dispersos    |
| laghoras    | Horas entre cierre y apertura de la orden de reparación    | 15,720    | Datos muy dispersos    |
| TOTAL    | Total de la reparación (Incluye IVA)   | 1,086,544   | Datos demasiado dispersos   |
| UnitYear    | Año del modelo de la unidad   | 116   | Datos no dispersos   |




*   **lagdias**: Los días de apertura y cierre de una reparación están ligeramente dispersos
*   **laghoras**: Las horas de apertura y cierre de una reparación presentan un gran rango de diferencia entre el valor máximo y mínimo, además de que están muy dispersas.
*   **TOTAL**: El costo total de las reparaciones tienen el rango más amplio, y con una alta desviación estándar.
*   **UnitYear**: Al tratarse de los años del modelo de cada unidad, no se presenta tanta dispersión.





In [ ]:
msno.matrix(data_descriptive[['lagdias', 'laghoras', 'TOTAL', 'UnitYear']])
#No hay notoria presencia de valores null

In [ ]:
data_descriptive.boxplot(figsize = (16, 8))
#El costo total de una reparación presenta la mayor cantidad de outliers, valores extremos
#Es importante consultar si estos valores son correctos, o pudiera tratarse de un error, de lo contrario se deberían eliminarse o imputarse

#El resto de variables no presentan tantos valores extremos

In [ ]:
#Histograma para analizar la variable más dispersa, TOTAL

data_descriptive.hist(['TOTAL'], figsize = (16,8))

**Estadística descripctiva de variables categóricas**

In [ ]:
# Estadística descripctiva de variables categóricas
data_descriptive.describe(include=object)

In [ ]:
# Valores más repetidos en las variables cualitativas
data_categorical = data_descriptive.select_dtypes(include=object)
for column in data_categorical.columns:
  print(data_categorical[column].value_counts().head(5))
  print()
# Borrar las variables temporales de la memoria
del column

Identificar si hay columnas comunes en los dataframes empleados que permitan “enlazar” los registros en las siguientes fases.

**Debido a que solo utilizamos un dataframe para esta parte inicial del proyecto, no contamos con columnas por las que se pueda enlazar a otros dataframes. A futuro sería posible que tengamos más de uno que sí valdría la pena enlazar a traves de llaves o columnas en común.**

# **Selección y limpieza de datos**

## Eliminar las columnas no relevantes

Columna|Justificacion
------|--------------
Column| No sabemos que es
Taller| Viene implicito en el OrderID
ClosedDate| Tenemos una metrica mas precisa (laghoras)
ClosedDay| ""
ClosedMonth| ""
ClosedYear| ""
OpenedDay| Tenemos una metrica mas general (OpenedDate)
OpenedMonth| ""
OpenedYear| ""
CreatedBy| No afecta realmente quien inicio la orden de reparacion
ClosedBy| No afecta realmente quien cerro la orden de reparacion
OrderType| Solo existen 2 tipos 'Standard' & 'Vendor'. Asumimos que no tiene efecto en la reparacion
Supplier| No afecta realmente de donde se obtuvo la pieza de reparacion
**COMPCDKEY**| **No sabemos si tiene utilidad o no**
**COMPCODE**| **No sabemos si tiene utilidad o no**
DESCRIP| Tiene informacion demasiado especifica para cada caso, no nos seria util
QTYRCVD| No afecta realmente si se daño 1 o mas piezas, si no, que fallaron
QTYREQD| Similar al anterior.
CHGAMT| Lo importante es la metrica del costo por reparacion (TOTAL)
TAXAMT| ""
SUBTOTAL| ""
EC| No afecta realmente el equipo administrativo del proyecto
ReasonType| Segun el diccionario de datos, no aporta informacion relevante
FechaPromesa| No genera impacto en la reparacion
Estimate| " "
CompleteDay| Tenemos una metrica mas precisa (laghoras)
CompleteMonth| ""
CompleteYear| ""
CompleteDate| ""
MECHANIC| Asumimos que no genera impacto en la reparacion quien la hizo

In [ ]:
# Seleccionar sólo las columnas relevantes para las estadisticas
data_filtered_cols = data.drop(['Column', 'Taller', 'ClosedDate', 'Status',
                   'ClosedDay', 'ClosedMonth', 'ClosedYear', 'OpenedDay', 'OpenedMonth', 'OpenedYear',
                   'CreatedBy', 'ClosedBy', 'OrderType', 'Supplier', 'COMPCDKEY', 'COMPCODE',
                   'QTYRCVD', 'QTYREQD', 'CHGAMT', 'TAXAMT', 'SUBTOTAL', 'EC',
                   'ReasonType', 'FechaPromesa', 'Estimate', 'CompleteDay',
                   'CompleteMonth', 'CompleteYear', 'CompleteDate', 'MECHANIC'], axis=1)

# Visualizacion de una muestra de los datos en el DataFrame
data_filtered_cols.sample(10)

## Limpieza de las columnas (Trim)

Notamos que algunas de las columnas extraídas desde el archivo .CSV tenían espacios en blanco. Dichos espacios deben ser removidos para evitar complicaciones más adelante en la agrupación y filtrado de información. Es por eso que procedemos a arreglar dichos errores en la captura de la información

In [ ]:
# Visualizacion de la informacion del DataFrame
data_filtered_cols.info()

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'RepReason'
data_filtered_cols['RepReason'] = data_filtered_cols['RepReason'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['RepReason'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'UnitType'
data_filtered_cols['UnitType'] = data_filtered_cols['UnitType'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['UnitType'].value_counts().index

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'TipoLinea'
data_filtered_cols['Flota'] = data_filtered_cols['Flota'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['Flota'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'COMPLAINT'
data_filtered_cols['COMPLAINT'] = data_filtered_cols['COMPLAINT'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['COMPLAINT'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'Jobcode'
data_filtered_cols['Jobcode'] = data_filtered_cols['Jobcode'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['Jobcode'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'TipoLinea'
data_filtered_cols['TipoLinea'] = data_filtered_cols['TipoLinea'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['TipoLinea'].value_counts().index

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'NumParte'
data_filtered_cols['NumParte'] = data_filtered_cols['NumParte'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['NumParte'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'CITA'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['CITA'].value_counts().index

Para la ultima columna 'CITA' se observa que existen registros con un campo vacio, por lo que se procede a sustituir este campo N/A por un texto de referencia (DESCONOCIDO). Además podemos corregir los nombres de algunos tipos de cita: 

* 'CITA A TIEMP' -> 'CITA A TIEMPO'
* 'EXPRES' -> 'EXPRESS'
* 'SCHEDULED' -> 'CON CITA'

In [ ]:
# Sustituir valores de 'CITA' con un string vacio por 'Desconocido'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('', 'DESCONOCIDO')

# Sustituir valores de 'CITA A TIEMP' por 'CITA A TIEMPO'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('CITA A TIEMP', 'CITA A TIEMPO')

# Sustituir valores de 'EXPRES' por 'EXPRESS'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('EXPRES', 'EXPRESS')

# Sustituir valores de 'SCHEDULED' por 'CON CITA'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('SCHEDULED', 'CON CITA')

# Verificar que se haya hecho correctamente
data_filtered_cols['CITA'].value_counts().index

## Limpieza de columnas (DESC)

Para esta columna, necesitamos asegurarnos que todas las palabras vengan en mayusculas, sin espacios en blanco al inicio y al final y además aegurarnos que no haya valores faltantes

In [ ]:
# Modificar nombre de columna 'DESCRIP' por 'DESCRIPTION'
data_filtered_cols.rename(columns={'DESCRIP': 'DESCRIPTION'}, inplace=True)

# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'DESCRIPTION'
data_filtered_cols['DESCRIPTION'] = data_filtered_cols['DESCRIPTION'].str.strip()

# Convertir la columna 'DESCRIPTION' a mayusculas
data_filtered_cols['DESCRIPTION'] = data_filtered_cols['DESCRIPTION'].str.upper()

# Verificar que la columna no tenga valores nulos
print(f'Cantidad de valores nulos en la columna DESCRIPTION: {data_filtered_cols["DESCRIPTION"].isnull().sum()}')

## Restablecer los índices con identificadores adecuados.

El índice del dataframe ya está establecido, considerando evitar los valores duplicados.

## Seleccionar o filtrar los registros u observaciones de interés
Con la finalidad de operar dataframes de menor complejidad.

Dado el proyecto asignado al equipo, nos centraremos en la informacion sobre todas las reparaciones de la flota 'Sayer', por lo que deberemos filtrar dicha informacion.

In [ ]:
# Extraemos la informacion del proyecto Sayer Full
data_sayer = data_filtered_cols[data_filtered_cols['Flota'] == 'Sayer Full'][:]
# Borrar la columna 'Flota', pues ya no es necesaria
data_sayer = data_sayer.drop('Flota', axis=1)
data_sayer.info()

In [ ]:
# Eliminar dataFrames que ya no se utilizaran
del data_descriptive, data_categorical, data_filtered_cols, columns

## Identificar valores faltantes y eliminarlos

In [ ]:
# Visualización de datos nulos en el DataFrame
data_sayer.isnull().sum()

In [ ]:
# Se identifican valores nulos en las columnas 'kMS', 'TipoLinea', 'CITA' y 'NumParte'
data_sayer[['kMS', 'TipoLinea', 'CITA', 'NumParte']].isnull().sum()

In [ ]:
# Visualizacion de datos nulos en el DataFrame
data_sayer[data_sayer.isnull().any(axis=1)]

In [ ]:
# Sustituir valores nulos de la columna 'CITA' por 'DESCONOCIDO'
data_sayer['CITA'] = data_sayer['CITA'].fillna('DESCONOCIDO')
data_sayer['CITA'].isnull().sum()

In [ ]:
# Sustituir valores nulos de la columna 'TipoLinea' por 'DESCONOCIDO'
data_sayer['TipoLinea'] = data_sayer['TipoLinea'].fillna('DESCONOCIDO')
data_sayer['TipoLinea'].isnull().sum()

In [ ]:
# Sustituir valores nulos de la columna 'NumParte' por 'DESCONOCIDO'
data_sayer['NumParte'] = data_sayer['NumParte'].fillna('DESCONOCIDO')
data_sayer['NumParte'].isnull().sum()

In [ ]:
# Verificar que se hayan sustituido correctamente los valores nulos
data_sayer.isnull().sum()

## Detectar valores atípicos para su posterior transformación.


In [ ]:
data_sayer.boxplot(vert=False)
#Observamos que en la columna de TOTAL es la que aparentemente hay mas datos
#atipicos, por lo que vamos a anilizarla a detalle


In [ ]:
# Información estadística del DataFrame
data_sayer.describe()

In [ ]:
# Análisis de la columna 'TOTAL'
data_sayer[['TOTAL']].boxplot(vert=False)

In [ ]:
# Visualizacion de datos atipicos en la columna 'TOTAL'
data_sayer[data_sayer['TOTAL'] > 30000]

In [ ]:
# Máscaras para identificar valores atipicos en la columna 'TOTAL'

# Consideramos que no es congruente que la columna 'TOTAL' tenga valores negativos
sayer_total_atipicosBajos = data_sayer['TOTAL'] <= 0

# Consideramos que es posible que una reparacion sea bastante costosa. Es por ello que 
# consideramos mantener los gastos elevados, exceptuando aquellos que están muy alejados 
# que podemos calcular en gastos superiores a $30,000
sayer_total_atipicosAltos = data_sayer['TOTAL'] >= 30000

# Concatenar las mascaras para obtener los valores atipicos
valores_atipicos = sayer_total_atipicosBajos | sayer_total_atipicosAltos

# Conteo de valores atipicos
valores_atipicos.sum()

In [ ]:
# Creación de un nuevo DataFrame sin los valores atipicos
data_sayer_clean = data_sayer[~valores_atipicos]

In [ ]:
# Análisis de la columna 'TOTAL' sin valores atipicos
data_sayer_clean[['TOTAL']].boxplot(vert=False)

In [ ]:
# Análisis de la columna 'laghoras'
data_sayer_clean[['laghoras']].boxplot(vert=False)

In [ ]:
# Visualizacion de datos atipicos en la columna 'laghoras'
data_sayer_clean[data_sayer_clean['laghoras'] < 0]

In [ ]:
# Máscaras para identificar valores atipicos en la columna 'laghoras'

# Consideramos que no es congruente que la columna 'laghoras' tenga valores negativos o en 0
sayer_laghoras_atipicosBajos = data_sayer_clean['laghoras'] <= 0

# Consideramos que es posible que una reparacion sea bastante tardada. Es por ello que
# consideramos mantener los tiempos elevados, exceptuando aquellos que están muy alejados
# que podemos calcular en tiempos superiores a 1500 horas (62.5 dias)
sayer_laghoras_atipicosAltos = data_sayer_clean['laghoras'] >= 1500

# Concatenar las mascaras para obtener los valores atipicos
valores_atipicos = sayer_laghoras_atipicosBajos | sayer_laghoras_atipicosAltos

# Conteo de valores atipicos
valores_atipicos.sum()

In [ ]:
# Actualizacion del DataFrame sin los valores atipicos
data_sayer_clean = data_sayer_clean[~valores_atipicos]

In [ ]:
# Análisis de la columna 'laghoras' sin valores atipicos
data_sayer_clean[['laghoras']].boxplot(vert=False)

In [ ]:
# Análisis de la columna 'lagdias'. Como podemos observar, al imputar la columna 'laghoras'
# también se imputa la columna 'lagdias' ya que ambas eran un cálculo de la diferencia entre las fechas
data_sayer_clean[['lagdias']].boxplot(vert=False)

In [ ]:
# Visualizacion de datos atipicos en la columna 'lagdias'
data_sayer_clean[['kMS']].boxplot(vert=False)

In [ ]:
# Recordemos que la columna 'kMS' es la que contiene valores nulos que todavia no hemos tratado
# Aqui procedemos a analizar los valores atipicos en la columna 'kMS' y posteriormente tratar los valores
# por medio de una imputacion de la mediana
sayer_kMS_atipicosBajos = data_sayer_clean['kMS'].isna()

# Concatenar las mascaras para obtener los valores atipicos
valores_atipicos = sayer_kMS_atipicosBajos

# Conteo de valores atipicos
valores_atipicos.sum()

In [ ]:
# Actualizacion del DataFrame sin los valores atipicos
data_sayer_clean = data_sayer_clean.fillna(data_sayer_clean['kMS'].median())
data_sayer_clean.info()

In [ ]:
# Análisis del DataFrame original de 'data_sayer' sin valores atipicos
data_sayer = data_sayer_clean

In [ ]:
# Eliminacion de las variables temporales utilizadas para los valores atipicos
del sayer_total_atipicosBajos, sayer_total_atipicosAltos, sayer_laghoras_atipicosBajos 
del sayer_laghoras_atipicosAltos, sayer_kMS_atipicosBajos, valores_atipicos, data_sayer_clean

## Eliminar registros duplicados.

In [ ]:
# Cantidad de elementos en el df
data_sayer.index.size

In [ ]:
# Cantidad de elementos en el df si se eliminan las filas con informacion identica
data_sayer.drop_duplicates().index.size

Haciendo una resta simple, podemos calcular que actualmente existen $6378-6086 = 292$ duplicados. Esto añade un gran sesgo al análisis posterior, por lo que se procede a eliminarlos.

In [ ]:
# Eliminacion de filas con informacion identica
data_sayer.drop_duplicates(inplace=True)

# Resetear el indice del DataFrame
data_sayer.reset_index(drop=True, inplace=True)

In [ ]:
# Informacion del df despues de eliminar filas con informacion identica
data_sayer.info()

# **Construcción y transformación de datos**

## Agregación y agrupamiento.
 Construir nuevas características o variables con las tecnicas descritas

Primero, vamos a agrupar los valores de la columna 'COMPLAINT' para separar los siguientes tipos de
mantenimiento:
* PREVENTIVO
* CORRECTIVO
* DESCONOCIDO

In [ ]:
#Analizar los tipos de reparaciones que se realizan a los tractos más comunes
data_sayer['COMPLAINT'].value_counts().index

In [ ]:
# Añadir una nueva columna para identificar claramente los tipos de 'COMPLAINT'
# El valor por defecto de la columna sera 'CORRECTIVO' debido a que es el tipo de reparacion mas comun
data_sayer['ComplaintType'] = 'CORRECTIVO'

# Establecer en 'COMPLAINT' = 'PREVENTIVO' para los casos especficos que se pueden determinar
ComplaintTypePreventive = ['PM', 'PREVENT VENC', 'GASTADO', 'DESGASTADO', 'SUCIO', 'BAJA PRESION', 'PRESIÓN BAJA', 'GOLPETEA']
data_sayer.loc[data_sayer['COMPLAINT'].isin(ComplaintTypePreventive), 'ComplaintType'] = 'PREVENTIVO'

# Excepto para los casos que no se pueden determinar por ser demasiado ambiguos.
# Estos se clasificarán como 'DESCONOCIDO'
ComplaintTypeUnknown = ['DAÑO OP', 'SE NEUTRALIZ', 'DESGASTE IRR', 'JALA DERECHA', 'SIN DIESEL', 'ACORTADO', 'OTROS', 'OTHER']
data_sayer.loc[data_sayer['COMPLAINT'].isin(ComplaintTypeUnknown), 'ComplaintType'] = 'DESCONOCIDO'

In [ ]:
# Muestreamos para verificar que el agrupamiento se haya hecho correctamente
data_sayer['ComplaintType'].value_counts()

In [ ]:
#¿Qué porcentaje representan los mantenimiento preventivos segun el ComplaintType respecto a todas las operaciones de mantenimiento?
percent = (data_sayer.groupby('ComplaintType')['ComplaintType'].count() / data_sayer.shape[0] * 100).round(2)
# Ordenar por porcentaje
percent = percent.sort_values(ascending=False)

# Imprimir los porcentajes de mantenimientos por tipo
for i in range(len(percent)):
    print(f" (Complaint Type) Los mantenimientos de tipo '{percent.index[i]}' representan el {percent.iloc[i]}% de todas las operaciones de mantenimiento.")


Posteriormente, vamos a agrupar los valores de la columna 'RepReason' para identificar los tipos de
mantenimiento:
* PREVENTIVO
* CORRECTIVO
* DESCONOCIDO

In [ ]:
#Analizar los tipos de reparaciones que se realizan a los tractos más comunes
data_sayer['RepReason'].value_counts().index

In [ ]:
# Añadir una nueva columna para identificar claramente los tipos de 'COMPLAINT'
# El valor por defecto de la columna sera 'DESCONOCIDO' debido a que es el tipo de reparacion mas comun
# Lamentablemente, es muy ambigua la informacion en esta columna, por lo que tendremos que clasificarla como 'DESCONOCIDO'
data_sayer['RepReasonType'] = 'DESCONOCIDO'

# Establecer en 'COMPLAINT' = 'PREVENTIVO' para los casos especificos que se pueden determinar
RepReasonTypePreventive = ['PM', 'DESGASTE', 'LAVADO', 'FALTA PRESIÓ']
data_sayer.loc[data_sayer['RepReason'].isin(RepReasonTypePreventive), 'RepReasonType'] = 'PREVENTIVO'

# Establecer en 'COMPLAINT' = 'CORRECTIVO' para los casos especificos que se pueden determinar
RepReasonTypeCorrective = ['MC', 'PERFORADA', 'PONCHADURA', 'FUGA AIRE', 'FOCOS', 'FUGA ACEITE', 'CAMBIO DAÑO', 'ROBO', '7 VIAS DAÑAD', 'BATERÍA DAÑA', 'GOLPE', 'CORTO', 'FALLA GPS']
data_sayer.loc[data_sayer['RepReason'].isin(RepReasonTypeCorrective), 'RepReasonType'] = 'CORRECTIVO'

In [ ]:
# Muestreamos para verificar que el agrupamiento se haya hecho correctamente
data_sayer['RepReasonType'].value_counts()

In [ ]:
#¿Qué porcentaje representan los mantenimiento preventivos segun el ComplaintType respecto a todas las operaciones de mantenimiento?
percent = (data_sayer.groupby('RepReasonType')['RepReasonType'].count() / data_sayer.shape[0] * 100).round(2)
# Ordenar por porcentaje
percent = percent.sort_values(ascending=False)

# Imprimir los porcentajes de mantenimientos por tipo
for i in range(len(percent)):
    print(f" (RepReason Type) Los mantenimientos de tipo '{percent.index[i]}' representan el {percent.iloc[i]}% de todas las operaciones de mantenimiento.")

In [ ]:
# Eliminacion de las variables temporales utilizadas para categorizar los tipos de reparaciones
del ComplaintTypePreventive, ComplaintTypeUnknown, percent, i
del RepReasonTypePreventive, RepReasonTypeCorrective

También necesitamos agrupar la columna `DESCRIPTION` pues nos permitirá identificar los sistemas involucrados en las reparaciones para un posterior análisis de dichas reparaciones por sistema.

In [ ]:
# Obtener todos los valores unicos de la columna 'DESCRIPTION'
uniqueDescriptions = pd.Series(sorted(list(data_sayer['DESCRIPTION'].unique())))

# Clasificar por los distintos sistemas:

sistemaRemolque = ['QUINTA RUEDA', 'HEMBRA']

estructura = ['XRP', 'REMACHE', 'ABRAZADERA', 'SOLDA', 'TUERCA', 'LAVADO', 'TORNILL', 'LVULA', 
              'RONDANA', 'BISAGRA', 'CONE']

sistemaDePropulsion = ['COLLARIN', 'COMBUSTIBLE', 'TRANSMIS', 'MOTOR', 'MARCHA']

sistemaDeControl = ['ZAPATA', 'LLANTA', 'LANTA', 'RIN', 'PARCHE', 'TALACHA', 'SUSPENSI', 'FRENO ', 
                    'EJE', 'BALERO', 'BALATA','PONCHAD', 'RUEDA', 'AMORTI', 'PERNO', 'NEUM', 'ROTO', 
                    'BIRLO', 'BUJE', 'MATRACA', 'DIRECCI', 'JUNTA', 'HIDRA']

sistemaTransversalMotor = ['ACEITE', 'AIRE', 'REFRIGERANTE', 'AGUA', 'MANGUERA', 'RADIADOR', 'UREA', 
                           'VENTILADOR', 'FAN']

sistemaAC = ['AIRE ACONDICIONADO', 'A/C']

sistemaElectrico = ['BATER', 'FOCO', 'LUZ', 'LUCE', 'CABLE', 'FUSIBLE', 'LED', 'FARO', 'PLAF', 
                    'ELECTR', 'GPS', 'TORRETA']

miscelaneo = ['AFLOJATODO', 'BROCA', 'BROCHA', 'CHAVETA','CINCHO', 'CINTA','DISCO', 'FLEX', 'LODERA',
              'MAN', 'PUNTA', 'TAP', 'LIMPIA','SELLADOR', 'SOR', 'ADHESIVO', 'ANTICO', 'GRASA', 
              'LUBRICANTE', 'MAZA', 'PIJA', 'PINTURA']

sistems = [sistemaRemolque, estructura, sistemaDePropulsion, sistemaDeControl, sistemaTransversalMotor,
           sistemaAC, sistemaElectrico, miscelaneo]
sistems_names = ['Sistema Remolque', 'Estructura', 'Sistema de Propulsión', 'Sistema de Control', 
                 'Sistema Transversal Motor', 'Sistema AC', 'Sistema Electrico', 'Misceláneo']

# Crear una nueva columna para clasificar los sistemas de las descripciones
data_sayer['DescriptionType'] = 'DESCONOCIDO'

# Clasificar las descripciones en los sistemas correspondientes, asignandoles su sistema a la columna 'DescriptionType'
for sistem in sistems:
    for description in sistem:
        data_sayer.loc[data_sayer['DESCRIPTION'].str.contains(description), 'DescriptionType'] = sistems_names[sistems.index(sistem)]

# Eliminacion de las variables temporales utilizadas para clasificar las descripciones
del uniqueDescriptions

del sistemaRemolque, estructura, sistemaDePropulsion, sistemaDeControl, sistemaTransversalMotor
del sistemaAC, sistemaElectrico, miscelaneo

del sistems, sistems_names, sistem, description

## MTBF
The average time elapsed between consecutive failures.

In [ ]:
data_sayer.info()

In [ ]:
# Creacion de un DataFrame para el registro de reparaciones y fechas por cada unidad de la flota
# de Sayer Full

# Se agrupara la informacion por 'UnitID', 'UnitType'
# Se agrega la columna 'RepairCount' que obtiene la cantidad de reparaciones de una misma unidad
# Se agrega la columna 'RepairDates' que obtiene las fechas de las reparaciones de una misma unidad
sayer_maintenance_byUnit = data_sayer.groupby(['UnitID', 'UnitType']).agg(
    RepairCount=('OpenedDate', 'size'),
    RepairDates=('OpenedDate', lambda x: list(x + pd.to_timedelta(data_sayer.loc[x.index, 'laghoras'], unit='h')))
).reset_index()

# Muestreamos para verificar que el agrupamiento se haya hecho correctamente
sayer_maintenance_byUnit

In [ ]:
# Creacion de un DataFrame para el registro de reparaciones y fechas por cada unidad de la flota
# de Sayer Full por tipo de reparacion

# Se agrupara la informacion por 'UnitID' y UnitType
# Se agrega la columna 'RepairCount' que obtiene la cantidad de reparaciones de una misma unidad
# Se agrega la columna 'RepairDates' que obtiene las fechas de las reparaciones de una misma unidad
sayer_maintenance_byUnit_byRepReason = data_sayer.groupby(['UnitID', 'UnitType', 'RepReasonType']).agg(
    RepairCount=('OpenedDate', 'size'),
    RepairDates=('OpenedDate', lambda x: list(
        x + pd.to_timedelta(data_sayer.loc[x.index, 'laghoras'], unit='h')))
).reset_index()

# Muestreamos para verificar que el agrupamiento se haya hecho correctamente
sayer_maintenance_byUnit_byRepReason

In [ ]:
# Asegurarse de que las fechas estén en formato datetime y manejar errores
sayer_maintenance_byUnit['RepairDates'] = sayer_maintenance_byUnit['RepairDates'].apply(lambda x: pd.to_datetime(x))

# Asegurarse de que las fechas estén en formato datetime y manejar errores
sayer_maintenance_byUnit_byRepReason['RepairDates'] = sayer_maintenance_byUnit_byRepReason['RepairDates'].apply(lambda x: pd.to_datetime(x))

In [ ]:
# Función para calcular el promedio de días entre reparaciones
def meanBtwnRepairs(dates):
    if len(dates) > 1:
        dates = sorted(dates)  # Ordenar las fechas
        # Calcular diferencias entre fechas consecutivas y regresar el promedio en días
        return pd.Series(dates).diff().mean().days
    else:
        return None  # Si solo hay una fecha


# Función para calcular el promedio de días entre reparaciones
def meanBtwnRepairsHours(dates):
    if len(dates) > 1:
        dates = sorted(dates)  # Ordenar las fechas
        # Calcular diferencias entre fechas consecutivas y regresar el promedio en horas
        return pd.Series(dates).diff().mean().total_seconds() / 3600
    else:
        return None  # Si solo hay una fecha

In [ ]:
# Crear una nueva columna para el promedio de días entre reparaciones
sayer_maintenance_byUnit['AvgDaysBetweenRepairs'] = sayer_maintenance_byUnit['RepairDates'].apply(meanBtwnRepairs)
# Crear una nueva columna para el promedio de horas entre reparaciones
sayer_maintenance_byUnit['AvgHoursBetweenRepairs'] = sayer_maintenance_byUnit['RepairDates'].apply(meanBtwnRepairsHours)

In [ ]:
# Crear una nueva columna para el promedio de días entre reparaciones por tipo de reparacion
sayer_maintenance_byUnit_byRepReason['AvgDaysBetweenRepairs'] = sayer_maintenance_byUnit_byRepReason['RepairDates'].apply(meanBtwnRepairs)
# Crear una nueva columna para el promedio de horas entre reparaciones por tipo de reparacion
sayer_maintenance_byUnit_byRepReason['AvgHoursBetweenRepairs'] = sayer_maintenance_byUnit_byRepReason['RepairDates'].apply(meanBtwnRepairsHours)

### Distribucion de los datos

In [ ]:
# Crear un nuevo DataFrame con las reparaciones correctivas
sayer_maintenance_byUnit_corrective = sayer_maintenance_byUnit_byRepReason[sayer_maintenance_byUnit_byRepReason['RepReasonType'] == 'CORRECTIVO'].copy()

# Crear un nuevo DataFrame con las reparaciones preventivas
sayer_maintenance_byUnit_preventive = sayer_maintenance_byUnit_byRepReason[sayer_maintenance_byUnit_byRepReason['RepReasonType'] == 'PREVENTIVO'].copy()

In [ ]:
# Crear un histograma de la distribución de los días promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(
    sayer_maintenance_byUnit_corrective['AvgHoursBetweenRepairs'].dropna(), kde=True)
plt.title('Distribución de los días promedio entre reparaciones correctivas')
plt.xlabel('Días')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
sayer_maintenance_byUnit_corrective['AvgHoursBetweenRepairsPOW'] = scaler.fit_transform(
    sayer_maintenance_byUnit_corrective['AvgHoursBetweenRepairs'].values.reshape(-1, 1))

# Crear un histograma de la distribución de los días promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(
    sayer_maintenance_byUnit_corrective['AvgHoursBetweenRepairsPOW'].dropna(), kde=True)
plt.title('Distribución de las horas promedio entre reparaciones. Normalizado')
plt.xlabel('Días')
plt.ylabel('Frecuencia')
plt.show()

## Discretización o binning
Reagrupar atributos de alta cardinalidad mediante los metodos descritos

In [ ]:
#Elegimos el metodo de discretizacion qcut ya que no tenemos rangos bien definidos
#de manera que con este metodo dividimos las variables en intervalos iguales,
#a lo cual tomamos 3 criterios, Reparacion_rapida, Reparacion_Normal y
#Reparacion_lenta con la finalidad de hacer mas descriptiva la variable de horas
# y tener nocion si la reparacion tomo demasiado tiempo o fue muy lenta
#Decidimos eligir la variable de horas, porque al analizar esta variable, observamos
#que es mas exacta que la variable dias, ya que esta ultima son dias enteros.

data_sayer['EficienciaRep'], cut_bin = pd.qcut(x=data_sayer['laghoras'],
                                               q=3,
                                               labels=['RAPIDA','NORMAL','LENTA'],
                                               retbins=True)
cut_bin

In [ ]:
data_sayer['EficienciaRep'].value_counts()
#Observamos que lo dividio casi exactamente en 3 partes iguales

In [ ]:
#Similar al caso anterior, al no tener unos rangos bien definidos,
#nos decidimos por el metodo de qcut para dividirlo en 3 partes,Costo_bajo
# Costo_normal, Costo_elevado .

data_sayer['ClasificacionCosto'], cut_bin = pd.qcut(x=data_sayer['TOTAL'],
                                    q=3,
                                    labels=['BAJO','NORMAL','ELEVADO'],
                                    retbins=True)
cut_bin

In [ ]:
# Verificar la cantidad de valores en cada categoria de 'ClasificacionCosto'
data_sayer['ClasificacionCosto'].value_counts()

In [ ]:
# Eliminacion de las variables temporales utilizadas para la clasificacion de las variables
del cut_bin

In [ ]:
# Visualizacion de la informacion del DataFrame
data_sayer.info()

## One-hot encoding
Para las variables **categóricas** que consideren necesarias.

In [ ]:
data_sayer.select_dtypes(include=['object', 'category'])

In [ ]:
# Crear un nuevo DataFrame con las columnas a utilizar para el encoding
sayer_encoding = data_sayer[['OrderID','UnitID','CITA', 'TipoLinea', 'UnitType']]

In [ ]:
# codificación por conteo
sayer_encoding = CountEncoder(cols=['CITA']).fit(sayer_encoding).transform(sayer_encoding)

In [ ]:
# codificación por OneHot
sayer_encoding = OneHotEncoder(cols=['TipoLinea']).fit(sayer_encoding).transform(sayer_encoding)

In [ ]:
# codificación por OneHot
sayer_encoding = OneHotEncoder(cols=['UnitType']).fit(sayer_encoding).transform(sayer_encoding)

In [ ]:
# Renombrar las columnas para una mejor comprensión
sayer_encoding.rename(columns={'UnitType_1': 'TRACTOR', 'UnitType_2': 'TRAILER', 'UnitType_3': 'DOLLY', 'TipoLinea_1': 'PART', 'TipoLinea_2': 'SERVICES', 'TipoLinea_3': 'LABOR'}, inplace = True)
# Resetear el índice
sayer_encoding.reset_index(drop=True, inplace=True)
# Visualizar el DataFrame
sayer_encoding

Para la parte de 'One-Hot Encoding' se decidio codificar unicamente 3 variables. 'CITA', 'TipoLinea', 'UnitType'.

Para las variables 'TipoLinea' y 'UnitType' se decidio codificar con OneHot ya que de esta manera será mucho mas facil encontrar y leer que tipo de unidad de esta manejando y que tipo de linea.

La variable 'CITA' se codificó con Count ya que de esta manera se podrá conocer la cantidad de registros por cada situación de las citas y así tener un mayor control.

## Feature transformation
Para las variables con valores atípicos identificados.

In [ ]:
# Verificamos que columnas vamos a trabajar
# (Deben ser numericas, para poder transformarlas)
data_sayer.info()

In [ ]:
# Analizamos cual es el valor minimo para cada una, es muy importante
# para obtener el metodo para transformar
data_sayer.describe()

In [ ]:
pd.plotting.scatter_matrix(data_sayer[['lagdias', 'laghoras', 'TOTAL', 'UnitYear']],
                           figsize=(16,8))
plt.show()

# Aqui debemos identificar aquellas que no tienen una distribucion normal, y que tenga sentidos aplicar
# una transformacion. En este caso, podriamos ocupar las variables 'dias, horas y TOTAL'

In [ ]:
# Dada la naturaleza de nuestros datos, no seria posible aplicar una 'Transformada
# Logaritmica', porque contamos con valores entre 0 y 1.
# Ademas si intentamos utilizar un metodo como el de 'Box-Cox', nos daria un error
# en la ejecucion, pues este solo permite valores mayores a 1.
# Es por ello que nuestro unico metodo de transformacion plausible para utilizar
# seria el de Yeo-johnson, que acepta un rango de valores de negativos, positivos
# y 0.

scaler = PowerTransformer(method='yeo-johnson')

pow_trans = scaler.fit_transform(data_sayer[['lagdias', 'laghoras', 'TOTAL']])
# creamos un DataFrame para guardar los valores transformados. Sera igual al de data_sayer, sustituyendo las columnas a transformar
sayer_featureTransform = data_sayer.copy()
# Sustituir las columnas a transformar por las transformadas
sayer_featureTransform['lagdias'] = pow_trans[:, 0]
sayer_featureTransform['laghoras'] = pow_trans[:, 1]
sayer_featureTransform['total'] = pow_trans[:, 2]
# Cambiamos el nombre de dichas columnas para que se identifiquen como transformadas
sayer_featureTransform.rename(columns={'lagdias': 'pow_lagdias', 'laghoras': 'pow_laghoras', 'TOTAL': 'pow_total'}, inplace=True)

In [ ]:
# Comparativa entre las distribuciones anteriores y posteriores a la transformacion
fig, ax = plt.subplots(1, 2)
ax[0].hist(data_sayer.lagdias)
ax[0].set_title('LAGDIAS')
ax[1].hist(np.log(sayer_featureTransform.pow_lagdias))
ax[1].set_title('Pow LAGDIAS')

In [ ]:
# Comparativa entre las distribuciones anteriores y posteriores a la transformacion
fig, ax = plt.subplots(1, 2)
ax[0].hist(data_sayer.laghoras)
ax[0].set_title('LAGHORAS')
ax[1].hist(np.log(sayer_featureTransform.pow_laghoras))
ax[1].set_title('Pow LAGHORAS')

In [ ]:
# Comparativa entre las distribuciones anteriores y posteriores a la transformacion
fig, ax = plt.subplots(1, 2)
ax[0].hist(data_sayer.TOTAL)
ax[0].set_title('TOTAL')
ax[1].hist(np.log(sayer_featureTransform.pow_total))
ax[1].set_title('Pow TOTAL')

In [ ]:
# Finalmente hacemos una comparativa con la correlacion entre variables anterior, para afirmar 
# el incremento entre la relacion entre una y otra
fig, ax = plt.subplots(1, 2, figsize=(16,8))
sns.heatmap(data_sayer[['lagdias', 'laghoras', 'TOTAL']].corr(), annot=True, ax=ax[0]).set_title('Original data')
sns.heatmap(sayer_featureTransform[['pow_lagdias', 'pow_laghoras', 'pow_total']].corr(), annot=True, ax=ax[1]).set_title('Pow Transformed data')

In [ ]:
# Eliminacion de las variables temporales utilizadas para la transformacion de las variables
del ax, fig, pow_trans, scaler

## Reordenamiento de Columnas

In [ ]:
# Reordenar las columnas del DataFrame
data_sayer = data_sayer[['OrderID', 'UnitID', 'UnitType', 'UnitYear', 'kMS', 'OpenedDate', 'lagdias', 
                         'laghoras', 'EficienciaRep', 'RepReason', 'RepReasonType', 'COMPLAINT', 
                         'ComplaintType', 'DESCRIPTION', 'DescriptionType', 'Jobcode', 'TipoLinea', 
                         'NumParte', 'TOTAL', 'ClasificacionCosto']]

# RESULTADOS OBTENIDOS

En este entregable se logró encontrar el porcentaje significativo del número de rescates en carretera respecto al total.

Además, se clasificaron las reparaciones realizadas acorde a su periodo de tiempo, y los costos de las mismas según su rango.

Por último, se trabajaron los valores atípicos afirmando el incremento entre la variabilidad de los valores importantes para el proyecto.

# Parte 2 (Análisis de los datos)

## Antiguedad de la flota

In [ ]:
# Promedio de edad por cada Unidad segun su UnitID
print(f'El promedio de edad de las unidades de tipo DOLLY es de: ', end='')
print(f"{2024 - data_sayer[data_sayer['UnitType'] == 'DOLLY'].groupby('UnitID')['UnitYear'].mean().mean()} años")
print(f'El promedio de edad de las unidades de tipo TRACTOR es de: ', end='')
print(f"{2024 - data_sayer[data_sayer['UnitType'] == 'TRACTOR'].groupby('UnitID')['UnitYear'].mean().mean()} años")
print(f'El promedio de edad de las unidades de tipo TRAILER es de: ', end='')
print(f"{2024 - data_sayer[data_sayer['UnitType'] == 'TRAILER'].groupby('UnitID')['UnitYear'].mean().mean():.2f} años")

## Datos por año

In [ ]:
# Crear columna 'OpenedTrimester' para clasificar los trimestres de apertura
data_sayer['OpenedTrimester'] = data_sayer['OpenedDate'].dt.quarter

In [ ]:
# Crear columna 'ClosedMonth' para identificar el mes de cierre de la reparacion}
# Se calcula mediante la sumna de OpenedDate y lagdias
data_sayer['ClosedMonth'] = (data_sayer['OpenedDate'] + pd.to_timedelta(data_sayer['lagdias'], unit='d')).dt.month

In [ ]:
# Creacion de una columna 'MaintenanceYear' para identificar el año de mantenimiento
data_sayer['MaintenanceYear'] = data_sayer['OpenedDate'].dt.year
# Análisis de la columna 'MaintenanceYear' y los años de mantenimiento
data_sayer['MaintenanceYear'].value_counts()

In [ ]:
# Creacion de un DataFrame para los mantenimientos del año 2022
data_sayer_2022 = data_sayer[data_sayer['MaintenanceYear'] == 2022]
data_sayer_2022.sample(5)

In [ ]:
# Creacion de un DataFrame para los mantenimientos del año 2023
data_sayer_2023 = data_sayer[data_sayer['MaintenanceYear'] == 2023]
data_sayer_2023.sample(5)

In [ ]:
# Creacion de un DataFrame para los mantenimientos del año 2024
data_sayer_2024 = data_sayer[data_sayer['MaintenanceYear'] == 2024]
data_sayer_2024.sample(5)

In [ ]:
# Borrado de la columna 'MaintenanceYear' de los DataFrames creados, pues ya no es necesaria
data_sayer_2022 = data_sayer_2022.drop('MaintenanceYear', axis=1)
data_sayer_2023 = data_sayer_2023.drop('MaintenanceYear', axis=1)
data_sayer_2024 = data_sayer_2024.drop('MaintenanceYear', axis=1)

## Estadística hasta el momento

### Muestras

In [ ]:
# Cantidad de elementos en la poblablacion (data)
print(f'Nuestra poblacion (toda la base de datos) es de \033[4m{data.index.size}\033[0m elementos')
# Cantida de elementos en la muestra (data_sayer)
print(f'Nuestra muestra (proyecto Sayer Full) es de \033[4m{data_sayer.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2022)
print(f'Nuestra muestra (Sayer 2022) es de \033[4m{data_sayer_2022.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2023)
print(f'Nuestra muestra (Sayer 2023) es de \033[4m{data_sayer_2023.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2024)
print(f'Nuestra muestra (Sayer 2024) es de \033[4m{data_sayer_2024.index.size}\033[0m elementos')
# Tipos de unidades en la muestra (data_sayer). Agrupar por tipo de unidad y contar cuantas distintas hay segun su UnitID
print(f'\nEn  Sayer Full, se cuenta con:')
print(data_sayer.groupby("UnitType")["UnitID"].nunique())

### Reparaciones

#### MTBR

In [ ]:
# Cantidad de reparaciones por unidad de la flota de Sayer Full
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mfrecuencia de reparaciones:\033[0m')
sayer_maintenance_byUnit[['UnitID', 'UnitType','RepairCount', 'AvgDaysBetweenRepairs', 'AvgHoursBetweenRepairs']]

#### Tipos de reparaciones

In [ ]:
# Cantidad de reparaciones segun el tipo de reparacion (ComplaintType)
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mdistribucion de reparaciones:\033[0m')
print(f"{data_sayer.groupby(['ComplaintType'])['OrderID'].count()}\n")
print(f"{data_sayer.groupby(['MaintenanceYear'])['OrderID'].count()}\n")
print(f"{data_sayer.groupby(['MaintenanceYear', 'ComplaintType'])['OrderID'].count()}\n")


#### MTBR por UnitType

In [ ]:
# Tiempo promedio entre reparaciones por 'UnitType' en la flota de Sayer Full
print(f'\nEn Sayer Full, se tiene el siguiente \033[4m\033[1mpromedio de tiempo entre reparaciones:\033[0m')
sayer_maintenance_byUnit.groupby('UnitType')['AvgHoursBetweenRepairs'].mean()

#### MTBR por ComplaintType

In [ ]:
# Tiempo promedio en reparacion segun el tipo de reparacion (ComplaintType)
print(f'\nEn Sayer Full, se tiene el siguiente \033[4m\033[1mpromedio de tiempo en reparacion:\033[0m')
data_sayer.groupby('ComplaintType')['laghoras'].mean()

#### Repairs per Month

In [ ]:
# Graficar la cantidad de reparaciones por mes para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)
# Tamaño de las graficas
figure.set_size_inches(16, 6)
# Grafica 2022
sns.countplot(data=data_sayer_2022, ax=axis[0], x='ClosedMonth', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por mes en 2022')
axis[0].set_xlabel('Mes')
axis[0].set_ylabel('Cantidad de reparaciones')
# Grafica 2023
sns.countplot(data=data_sayer_2023, ax=axis[1], x='ClosedMonth', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por mes en 2023')
axis[1].set_xlabel('Mes')
axis[1].set_ylabel('Cantidad de reparaciones')
# Grafica 2024
sns.countplot(data=data_sayer_2024, ax=axis[2], x='ClosedMonth', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por mes en 2024')
axis[2].set_xlabel('Mes')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (ComplaintType) en cada mes
sns.countplot(data=data_sayer_2022, ax=axis[0], x='ClosedMonth', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer_2023, ax=axis[1], x='ClosedMonth', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer_2024, ax=axis[2], x='ClosedMonth', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')

plt.show()

In [ ]:
# Graficar la cantidad de reparaciones por mes para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)
# Tamaño de las graficas
figure.set_size_inches(16, 6)
# Grafica 2022
sns.countplot(data=data_sayer_2022, ax=axis[0], x='ClosedMonth', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por mes en 2022')
axis[0].set_xlabel('Mes')
axis[0].set_ylabel('Cantidad de reparaciones')
# Grafica 2023
sns.countplot(data=data_sayer_2023, ax=axis[1], x='ClosedMonth', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por mes en 2023')
axis[1].set_xlabel('Mes')
axis[1].set_ylabel('Cantidad de reparaciones')
# Grafica 2024
sns.countplot(data=data_sayer_2024, ax=axis[2], x='ClosedMonth', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por mes en 2024')
axis[2].set_xlabel('Mes')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (RepReason) en cada mes
sns.countplot(data=data_sayer_2022, ax=axis[0], x='ClosedMonth', hue='RepReasonType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer_2023, ax=axis[1], x='ClosedMonth', hue='RepReasonType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer_2024, ax=axis[2], x='ClosedMonth', hue='RepReasonType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')

plt.show()

#### Repairs per Trimester

In [ ]:
# Graficar la cantidad de reparaciones por trimestre para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)

# Tamaño de las graficas
figure.set_size_inches(16, 6)

# Grafica 2022
sns.countplot(data=data_sayer_2022, ax=axis[0], x='OpenedTrimester', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por trimestre en 2022')
axis[0].set_xlabel('Trimestre')
axis[0].set_ylabel('Cantidad de reparaciones')

# Grafica 2023
sns.countplot(data=data_sayer_2023, ax=axis[1], x='OpenedTrimester', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por trimestre en 2023')
axis[1].set_xlabel('Trimestre')
axis[1].set_ylabel('Cantidad de reparaciones')

# Grafica 2024
sns.countplot(data=data_sayer_2024, ax=axis[2], x='OpenedTrimester', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por trimestre en 2024')
axis[2].set_xlabel('Trimestre')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (ComplaintType) en cada trimestre

sns.countplot(data=data_sayer_2022, ax=axis[0], x='OpenedTrimester', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer_2023, ax=axis[1], x='OpenedTrimester', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer_2024, ax=axis[2], x='OpenedTrimester', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')

plt.show()

In [ ]:
# Eliminacion de las variables temporales
del figure, axis

## Costos

### por UnitID

In [ ]:
# Calcular la suma del costo total de las reparaciones para cada unidad de la flota
print(f'Costo total de las reparaciones por unidad de la flota de Sayer Full:')
data_sayer.groupby(['UnitID', 'UnitType'])['TOTAL'].sum().reset_index()

### por UnitType

In [ ]:
# Costo promedio por tipo de unidad
print(f'Costo promedio de las reparaciones por tipo de unidad:')
data_sayer.groupby('UnitType')['TOTAL'].mean().reset_index()

### por UnitYear

In [ ]:
# Costo promedio por año de unidad
print(f'Costo promedio de las reparaciones por tipo de unidad:')
data_sayer.groupby(['UnitYear', 'UnitType'])['TOTAL'].mean().reset_index()

### por ComplaintType

In [ ]:
# Costo promedio por tipo de reparacion (ComplaintType)
print(f'Costo promedio de las reparaciones por tipo de reparacion:')
data_sayer.groupby('ComplaintType')['TOTAL'].mean().reset_index()

# Parte 3 (Hipótesis Estadística)

## Reparaciones

Por: UnitYear, UnitType, ComplaintType

In [ ]:
# Conteo de las reparaciones por año, tipo de unidad y tipo de reparacion a traves del conteo de registros
data_sayer.groupby(['UnitYear', 'UnitType', 'ComplaintType'])['UnitID'].count()

In [ ]:
# Guardar el agrupamiento en un nuevo DataFrame
sayer_analysis = data_sayer.groupby(['UnitYear', 'UnitType', 'ComplaintType'])['UnitID'].count().reset_index()

# Renombrar la columna 'UnitID' por 'RepairCount'
sayer_analysis.rename(columns={'UnitID': 'RepairCount'}, inplace=True)

# Porcentaje de reparaciones correctivas sobre el total de reparaciones por año y tipo de unidad
sayer_analysis['PercentOverTotal'] = sayer_analysis['RepairCount'] / sayer_analysis.groupby(['UnitYear', 'UnitType'])['RepairCount'].transform('sum')

print(f'Porcentaje de reparaciones correctivas de Trailers en 2016:')
print(f'{sayer_analysis.iloc[0, 4] * 100:.2f}%')

print(f'Porcentaje de reparaciones correctivas de Trailers en 2017:')
print(f'{sayer_analysis.iloc[3, 4] * 100:.2f}%')

print(f'Porcentaje de reparaciones correctivas de Trailers en 2018:')
print(f'{sayer_analysis.iloc[6, 4] * 100:.2f}%')

sayer_analysis.groupby(['UnitYear'])['RepairCount'].sum()

## MTTR

In [ ]:
# laghoras promedio por tipo de reparacion (ComplaintType)
print(f'Promedio de horas por tipo de reparacion:')
data_sayer.groupby('ComplaintType')['laghoras'].mean()

## Distribucion de los datos

### MTTR

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_horas = scaler.fit_transform(data_sayer['laghoras'].values.reshape(-1, 1))

In [ ]:
# Crear un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_horas.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Horas de reparacion" (MTTR)')
plt.show()

In [ ]:
# Crear un histograma de la distribución de los días promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(pow_horas, kde=True)
plt.title('Distribución de las "horas promedio" (MTTR) en reparaciones. Normalizado')
plt.xlabel('Horas')
plt.ylabel('Frecuencia')
plt.show()

### Costos

In [ ]:
# Histograma de la distribución de los costos de mantenimiento
plt.figure(figsize=(16, 8))
sns.histplot(data_sayer['TOTAL'], kde=True)
plt.title('Distribución de los costos de mantenimiento. RAW')
plt.xlabel('Costo')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_total = scaler.fit_transform(
    data_sayer['TOTAL'].values.reshape(-1, 1))

In [ ]:
# Crear un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_total.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento"')
plt.show()

In [ ]:
# Histograma de la distribución de los costos de mantenimiento
plt.figure(figsize=(16, 8))
sns.histplot(pow_total, kde=True)
plt.title('Distribución de los costos de mantenimiento. Normalizado')
plt.xlabel('Costo')
plt.ylabel('Frecuencia')
plt.show()

### Mantenimiento Correctivo

In [ ]:
# Frecuencias de mantenimiento correctivo por tipo de unidad (UnitType)
print(f'Frecuencia de mantenimiento correctivo por tipo de unidad:')
sayer_maintenance_byUnit_corrective.groupby('UnitType')['RepairCount'].sum()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(sayer_maintenance_byUnit_corrective['RepairCount'], kde=True)
plt.title('Distribución de las frecuencias de mantenimiento correctivo. RAW')
plt.xlabel('Frecuencia de mantenimiento correctivo')
plt.ylabel('Cantidad de unidades')


In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(
    sayer_maintenance_byUnit_corrective['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Creacion de un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_repaircount.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Frecuencia de mantenimiento correctivo"')
plt.show()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(pow_repaircount, kde=True)
plt.title('Distribución de las frecuencias de mantenimiento correctivo. Normalizado')
plt.xlabel('Frecuencia de mantenimiento correctivo.')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Eliminacion de las variables temporales utilizadas para la transformacion de las variables
del pow_horas, pow_total, pow_repaircount, scaler

# Parte 4 (Hand's On 3A)

In [ ]:
# Registros de data_sayer con ComplaintType 'PREVENTIVE'. Obtener el promedio de sus costos
sayer_maintenance_byUnit_preventive['MeanCost'] = data_sayer[data_sayer['ComplaintType'] == 'PREVENTIVO'].groupby('UnitID')['TOTAL'].mean().values

# Sacar la correlacion entre las variables 'RepaitCount' y 'TotalCost'
sayer_maintenance_byUnit_preventive[['RepairCount', 'MeanCost']].corr(method='spearman')

In [ ]:
# Grafica QQ-Plot para verificar la normalidad de los datos
stats.probplot(sayer_maintenance_byUnit_preventive['RepairCount'], dist="norm", plot=plt)
plt.title('QQ-Plot "Frequencia de mantenimiento preventivo"')
plt.show()

In [ ]:
# Grafica QQ-Plot para verificar la normalidad de los datos
stats.probplot(sayer_maintenance_byUnit_preventive['MeanCost'], dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento preventivo"')
plt.show()

In [ ]:
# Plot scatter plot
plt.figure(figsize=(16, 8))
sns.scatterplot(x='RepairCount', y='MeanCost', data=sayer_maintenance_byUnit_preventive)
plt.title('Correlación entre Frecuencia y Costo promedio de mantenimiento preventivo')
plt.xlabel('Frecuencia de mantenimiento preventivo')
plt.ylabel('Costo promedio de mantenimiento preventivo')
plt.show()

# **REGRESIÓN LINEAL**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from sklearn.metrics import root_mean_squared_error
import statsmodels.api as sm


# Dividir los datos en en train y test
X = sayer_maintenance_byUnit_preventive[['RepairCount']]  # Variable independiente
y = sayer_maintenance_byUnit_preventive['MeanCost']    # Variable dependiente

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.8,
                                        random_state = 42,
                                        shuffle      = True
                                    )

# Creación del modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X=X_train, y=y_train)

# Información del modelo
print(f"Intercept: {modelo.intercept_}")
print(f"Coeficiente: {list(zip(modelo.feature_names_in_, modelo.coef_))}")

# Evaluación en el conjunto de entrenamiento
y_train_pred = modelo.predict(X_train)
print(f"R^2 en train: {modelo.score(X_train, y_train)}")

# Evaluación en el conjunto de prueba
y_test_pred = modelo.predict(X_test)
print(f"R^2 en test: {r2_score(y_test, y_test_pred)}")
print(f"RMSE en test: {np.sqrt(mean_squared_error(y_test, y_test_pred))}")

# Crear el modelo con statsmodels para obtener un resumen detallado
X_train_sm = sm.add_constant(X_train)  # Agregar término de intersección
modelo_sm = sm.OLS(y_train, X_train_sm).fit()
print()
print(modelo_sm.summary())

Interpretación:

 ***Información del modelo***

*   **Intercept**: Cuando el número de reparaciones en un tracto es cero, el costo fijo asociado al mantenimiento de un tracto es de *776.50*.
*   **Coeficiente**: Por cada reparación adicional realizada en el tracto, el costo promedio de mantenimiento aumenta en aproximadamente *1.20* unidades monetarias.
*   Coeficiente de determinación 𝑅2: Mide qué tan bien el modelo explica la variabilidad de los datos.


In [ ]:
# MODELO DE REGRESIÓN 2
# Dividir los datos en train y test
X = sayer_maintenance_byUnit_preventive[['RepairCount']]  # Variable independiente
y = sayer_maintenance_byUnit_preventive['MeanCost']    # Variable dependiente

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.8,
                                        random_state = 42,
                                        shuffle      = True
                                    )

# Creación del modelo utilizando matrices como en scikit-learn
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.OLS(endog=y_train, exog=X_train)
modelo = modelo.fit()
print(modelo.summary())

In [ ]:
# Intervalos de confianza para los coeficientes del modelo
intervalos_ci = modelo.conf_int(alpha=0.05)
intervalos_ci.columns = ['2.5%', '97.5%']
intervalos_ci

In [ ]:
# Predicciones con intervalo de confianza del 95%
predicciones = modelo.get_prediction(exog=X_train).summary_frame(alpha=0.05)
predicciones.head()

In [ ]:
# Predicciones con intervalo de confianza del 95%
predicciones = modelo.get_prediction(exog=X_train).summary_frame(alpha=0.05)
predicciones['x'] = X_train.loc[:, 'RepairCount']
predicciones['y'] = y_train
predicciones = predicciones.sort_values('x')

# Gráfico del modelo
fig, ax = plt.subplots(figsize=(6, 3.84))

ax.scatter(predicciones['x'], predicciones['y'], marker='o', color = "gray")
ax.plot(predicciones['x'], predicciones["mean"], linestyle='-', label="OLS")
ax.plot(predicciones['x'], predicciones["mean_ci_lower"], linestyle='--', color='red', label="95% CI")
ax.plot(predicciones['x'], predicciones["mean_ci_upper"], linestyle='--', color='red')
ax.fill_between(predicciones['x'], predicciones["mean_ci_lower"], predicciones["mean_ci_upper"], alpha=0.3)
ax.legend();

In [ ]:
# Error de test del modelo
X_test = sm.add_constant(X_test, prepend=True)
predicciones = modelo.predict(exog=X_test)
rmse = root_mean_squared_error(y_true=y_test, y_pred=predicciones)
print(f"El error (rmse) de test es: {rmse}")

In [ ]:
# Eliminacion de las variables temporales
del X, X_train, X_test, y, y_train, y_test, y_train_pred, y_test_pred, 
del modelo, modelo_sm, X_train_sm, intervalos_ci, predicciones, rmse, fig, ax

# Reparando data_sayer (data_sayer2)

Debido a que tenemos multiples reparaciones por dia (En una misma entrada a taller se registran varias reparaciones) las frecuencias de reparacion no eran adecuadas (estaban sesgadas). 

Por ello, debemos agrupar todas las reparaciones correctivas / preventivas realizadas el mismo dia, para garantizar que solo haya un registro diario de reparaciones, con un costo adecuado y así poder determinar las frecuencias de forma adecuada

In [ ]:
# Agrupar el df 'data_sayer' por UnitID, OpenedDate y ComplaintType, sumando los costos
data_sayer2 = data_sayer.groupby(['UnitID', 'OpenedDate', 'ComplaintType'])['TOTAL'].sum().reset_index()

# Añadir lagdias a 'data_sayer2'
data_sayer2['lagdias'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'ComplaintType'])['lagdias'].mean().reset_index()['lagdias'].values

# Añadir laghoras a 'data_sayer2'
data_sayer2['laghoras'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'ComplaintType'])['laghoras'].mean().reset_index()['laghoras'].values

# Añadir UnitType a 'data_sayer2'
data_sayer2['UnitType'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'ComplaintType'])['UnitType'].first().reset_index()['UnitType'].values

# Añadir kms a 'data_sayer2'
data_sayer2['kMS'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'ComplaintType'])['kMS'].first().reset_index()['kMS'].values

# Añadir UnitYear a 'data_sayer2'
data_sayer2['UnitYear'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'ComplaintType'])['UnitYear'].first().reset_index()['UnitYear'].values

# Crear columna 'OpenedTrimester' para clasificar los trimestres de apertura
data_sayer2['OpenedTrimester'] = data_sayer2['OpenedDate'].dt.quarter

# Crear columna 'ClosedMonth' para identificar el mes de cierre de la reparacion}
# Se calcula mediante la sumna de OpenedDate y lagdias
data_sayer2['ClosedMonth'] = (data_sayer2['OpenedDate'] + pd.to_timedelta(data_sayer2['lagdias'], unit='d')).dt.month

# Creacion de una columna 'MaintenanceYear' para identificar el año de mantenimiento
data_sayer2['MaintenanceYear'] = data_sayer2['OpenedDate'].dt.year

# Reordenar las columnas como en data_sayer
data_sayer2 = data_sayer2[['UnitID', 'UnitType', 'UnitYear', 'kMS', 'OpenedDate','lagdias',
                           'laghoras', 'ComplaintType', 'TOTAL', 'OpenedTrimester', 'ClosedMonth', 'MaintenanceYear']]

## sayer_maint2

In [ ]:
# Creacion de un DataFrame para el registro de reparaciones y fechas por cada unidad de la flota
# de Sayer Full

# Se agrupara la informacion por 'UnitID', 'UnitType'
# Se agrega la columna 'RepairCount' que obtiene la cantidad de reparaciones de una misma unidad
# Se agrega la columna 'RepairDates' que obtiene las fechas de las reparaciones de una misma unidad
sayer_maint2_byUnit = data_sayer2.groupby(['UnitID', 'UnitType']).agg(
    RepairCount=('OpenedDate', 'size'),
    RepairDates=('OpenedDate', lambda x: list(
        x + pd.to_timedelta(data_sayer2.loc[x.index, 'laghoras'], unit='h')))
).reset_index()


# Asegurarse de que las fechas estén en formato datetime y manejar errores
sayer_maint2_byUnit['RepairDates'] = sayer_maint2_byUnit['RepairDates'].apply(
    lambda x: pd.to_datetime(x))

## sayer_maint2_RepReason

In [ ]:
# Creacion de un DataFrame para el registro de reparaciones y fechas por cada unidad de la flota
# de Sayer Full por tipo de reparacion

# Se agrupara la informacion por 'UnitID' y UnitType
# Se agrega la columna 'RepairCount' que obtiene la cantidad de reparaciones de una misma unidad
# Se agrega la columna 'RepairDates' que obtiene las fechas de las reparaciones de una misma unidad
sayer_maint2_byUnit_byRepReason = data_sayer2.groupby(['UnitID', 'UnitType', 'UnitYear', 'ComplaintType']).agg(
    RepairCount=('OpenedDate', 'size'),
    RepairDates=('OpenedDate', lambda x: list(
        x + pd.to_timedelta(data_sayer.loc[x.index, 'laghoras'], unit='h')))
).reset_index()

In [ ]:
# Crear una nueva columna para el promedio de días entre reparaciones
sayer_maint2_byUnit['AvgDaysBetweenRepairs'] = sayer_maint2_byUnit['RepairDates'].apply(
    meanBtwnRepairs)
# Crear una nueva columna para el promedio de horas entre reparaciones
sayer_maint2_byUnit['AvgHoursBetweenRepairs'] = sayer_maint2_byUnit['RepairDates'].apply(
    meanBtwnRepairsHours)

## sayer_maint2 corrective / preventive

In [ ]:
# Crear un nuevo DataFrame con las reparaciones correctivas
sayer_maint2_byUnit_corrective = sayer_maint2_byUnit_byRepReason[
    sayer_maint2_byUnit_byRepReason['ComplaintType'] == 'CORRECTIVO'].copy()

# Crear un nuevo DataFrame con las reparaciones preventivas
sayer_maint2_byUnit_preventive = sayer_maint2_byUnit_byRepReason[
    sayer_maint2_byUnit_byRepReason['ComplaintType'] == 'PREVENTIVO'].copy()

In [ ]:
# Eliminacion de las variables antiguas

del sayer_maintenance_byUnit, sayer_maintenance_byUnit_byRepReason
del sayer_maintenance_byUnit_corrective, sayer_maintenance_byUnit_preventive
del data_sayer_2022, data_sayer_2023, data_sayer_2024
del sayer_analysis, sayer_encoding, sayer_featureTransform

# ==============================================

# Datos por año

In [ ]:
# Creacion de un DataFrame para los mantenimientos del año 2022
data_sayer2_2022 = data_sayer2[data_sayer2['MaintenanceYear'] == 2022]

# Creacion de un DataFrame para los mantenimientos del año 2023
data_sayer2_2023 = data_sayer2[data_sayer2['MaintenanceYear'] == 2023]

# Creacion de un DataFrame para los mantenimientos del año 2024
data_sayer2_2024 = data_sayer2[data_sayer2['MaintenanceYear'] == 2024]

In [ ]:
# Borrado de la columna 'MaintenanceYear' de los DataFrames creados, pues ya no es necesaria
data_sayer2_2022 = data_sayer2_2022.drop('MaintenanceYear', axis=1)
data_sayer2_2023 = data_sayer2_2023.drop('MaintenanceYear', axis=1)
data_sayer2_2024 = data_sayer2_2024.drop('MaintenanceYear', axis=1)

# Estadística hasta el momento

## Muestras

In [ ]:
# Cantidad de elementos en la poblablacion (data)
print(f'Nuestra poblacion (toda la base de datos) es de \033[4m{data.index.size}\033[0m elementos')
# Cantida de elementos en la muestra (data_sayer)
print(f'Nuestra muestra (proyecto Sayer Full) es de \033[4m{data_sayer2.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2022)
print(f'Nuestra muestra (Sayer 2022) es de \033[4m{data_sayer2_2022.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2023)
print(f'Nuestra muestra (Sayer 2023) es de \033[4m{data_sayer2_2023.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2024)
print(f'Nuestra muestra (Sayer 2024) es de \033[4m{data_sayer2_2024.index.size}\033[0m elementos')
# Tipos de unidades en la muestra (data_sayer). Agrupar por tipo de unidad y contar cuantas distintas hay segun su UnitID
print(f'\nEn  Sayer Full, se cuenta con:')
print(data_sayer2.groupby("UnitType")["UnitID"].nunique())

## Reparaciones

### MTBR

In [ ]:
# Cantidad de reparaciones por unidad de la flota de Sayer Full
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mfrecuencia de reparaciones:\033[0m')
sayer_maint2_byUnit[['UnitID', 'UnitType','RepairCount', 'AvgDaysBetweenRepairs', 'AvgHoursBetweenRepairs']]

In [ ]:
# Cantidad de reparaciones segun el tipo de reparacion (ComplaintType)
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mdistribucion de reparaciones:\033[0m\n')
print(f'{data_sayer2["ComplaintType"].value_counts()}\n')
print(f'{data_sayer2["MaintenanceYear"].value_counts()}\n')
print(f"{data_sayer2.groupby(['MaintenanceYear', 'ComplaintType']).size()}")

#### MTBR por UnitType

In [ ]:
# Tiempo promedio entre reparaciones por 'UnitType' en la flota de Sayer Full
print(f'\nEn Sayer Full, se tiene el siguiente \033[4m\033[1mpromedio de tiempo entre reparaciones:\033[0m\n')
print(f"{sayer_maint2_byUnit.groupby('UnitType')['AvgHoursBetweenRepairs'].mean()}")

#### MTBR por ComplaintType

In [ ]:
# Tiempo promedio en reparacion segun el tipo de reparacion (ComplaintType)
print(f'\nEn Sayer Full, se tiene el siguiente \033[4m\033[1mpromedio de tiempo en reparacion:\033[0m\n')
print(f"{data_sayer.groupby('ComplaintType')['laghoras'].mean()}")

### Distribucion de reparaciones

#### Repairs per Month

In [ ]:
# Graficar la cantidad de reparaciones por mes para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)
# Tamaño de las graficas
figure.set_size_inches(16, 6)
# Grafica 2022
sns.countplot(data=data_sayer2_2022, ax=axis[0], x='ClosedMonth', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por mes en 2022')
axis[0].set_xlabel('Mes')
axis[0].set_ylabel('Cantidad de reparaciones')
# Grafica 2023
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='ClosedMonth', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por mes en 2023')
axis[1].set_xlabel('Mes')
axis[1].set_ylabel('Cantidad de reparaciones')
# Grafica 2024
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='ClosedMonth', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por mes en 2024')
axis[2].set_xlabel('Mes')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (ComplaintType) en cada mes
sns.countplot(data=data_sayer2_2022, ax=axis[0], x='ClosedMonth', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='ClosedMonth', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='ClosedMonth', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')

plt.show()

#### Repairs per Trimester

In [ ]:
# Graficar la cantidad de reparaciones por trimestre para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)

# Tamaño de las graficas
figure.set_size_inches(16, 6)

# Grafica 2022
sns.countplot(data=data_sayer2_2022, ax=axis[0], x='OpenedTrimester', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por trimestre en 2022')
axis[0].set_xlabel('Trimestre')
axis[0].set_ylabel('Cantidad de reparaciones')

# Grafica 2023
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='OpenedTrimester', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por trimestre en 2023')
axis[1].set_xlabel('Trimestre')
axis[1].set_ylabel('Cantidad de reparaciones')

# Grafica 2024
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='OpenedTrimester', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por trimestre en 2024')
axis[2].set_xlabel('Trimestre')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (ComplaintType) en cada trimestre

sns.countplot(data=data_sayer2_2022, ax=axis[0], x='OpenedTrimester', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='OpenedTrimester', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='OpenedTrimester', hue='ComplaintType', hue_order=['CORRECTIVO', 'PREVENTIVO', 'DESCONOCIDO'], palette='Accent_r')

plt.show()

In [ ]:
# Eliminacion de las variables temporales
del figure, axis

## Costos

### por UnitID

In [ ]:
# Calcular la suma del costo total de las reparaciones para cada unidad de la flota
print(f'Costo total de las reparaciones por unidad de la flota de Sayer Full:')
data_sayer2.groupby(['UnitID', 'UnitType'])['TOTAL'].sum().reset_index()

### por UnitType

In [ ]:
# Costo promedio por tipo de unidad
print(f'Costo promedio de las reparaciones por tipo de unidad:')
data_sayer2.groupby('UnitType')['TOTAL'].mean().reset_index()

### por UnitYear

In [ ]:
# Costo promedio por año de unidad
print(f'Costo promedio de las reparaciones por tipo de unidad:')
data_sayer2.groupby(['UnitYear', 'UnitType'])['TOTAL'].mean().reset_index()

### por ComplaintType

In [ ]:
# Costo promedio por tipo de reparacion (ComplaintType)
print(f'Costo promedio de las reparaciones por tipo de reparacion:')
data_sayer2.groupby('ComplaintType')['TOTAL'].mean().reset_index()

# Pruebas de Hipótesis

## Reparaciones

Por: UnitYear, UnitType, ComplaintType

In [ ]:
# Conteo de las reparaciones por año, tipo de unidad y tipo de reparacion a traves del conteo de registros
data_sayer2.groupby(['UnitYear', 'UnitType', 'ComplaintType'])['UnitID'].count()

In [ ]:
# Guardar el agrupamiento en un nuevo DataFrame
sayer2_analysis = data_sayer2.groupby(['UnitYear', 'UnitType', 'ComplaintType'])['UnitID'].count().reset_index()

# Renombrar la columna 'UnitID' por 'RepairCount'
sayer2_analysis.rename(columns={'UnitID': 'RepairCount'}, inplace=True)

# Porcentaje de reparaciones correctivas sobre el total de reparaciones por año y tipo de unidad
sayer2_analysis['PercentOverTotal'] = sayer2_analysis['RepairCount'] / sayer2_analysis.groupby(['UnitYear', 'UnitType'])['RepairCount'].transform('sum')

print(f'Porcentaje de reparaciones correctivas de Trailers en 2016:')
print(f'{sayer2_analysis.iloc[0, 4] * 100:.2f}%')

print(f'Porcentaje de reparaciones correctivas de Trailers en 2017:')
print(f'{sayer2_analysis.iloc[3, 4] * 100:.2f}%')

print(f'Porcentaje de reparaciones correctivas de Trailers en 2018:')
print(f'{sayer2_analysis.iloc[6, 4] * 100:.2f}%')

print(f"\n{sayer2_analysis.groupby(['UnitYear'])['RepairCount'].sum()}")

## MTTR

In [ ]:
# laghoras promedio por tipo de reparacion (ComplaintType)
print(f'Promedio de horas por tipo de reparacion:')
print(f"\n{data_sayer2.groupby('ComplaintType')['laghoras'].mean()}")

## Distribucion de los datos

### MTTR

In [ ]:
# Crear un histograma de la distribución de las horas promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(data_sayer2['laghoras'], kde=True)
plt.title('Distribución de las "horas promedio" (MTTR) en reparaciones. RAW')
plt.xlabel('Horas')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_horas = scaler.fit_transform(data_sayer2['laghoras'].values.reshape(-1, 1))

In [ ]:
# Crear un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_horas.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Horas de reparacion" (MTTR)')
plt.show()

In [ ]:
# Crear un histograma de la distribución de las horas promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(pow_horas, kde=True)
plt.title('Distribución de las "horas promedio" (MTTR) en reparaciones. Normalizado')
plt.xlabel('Horas')
plt.ylabel('Frecuencia')
plt.show()

### Costos

In [ ]:
# Histograma de la distribución de los costos de mantenimiento
plt.figure(figsize=(16, 8))
sns.histplot(data_sayer2['TOTAL'], kde=True)
plt.title('Distribución de los costos de mantenimiento. RAW')
plt.xlabel('Costo')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_total = scaler.fit_transform(data_sayer2['TOTAL'].values.reshape(-1, 1))

In [ ]:
# Asignar la transformacion a una nueva columna
data_sayer2['TOTAL_norm'] = pow_total

In [ ]:
# Crear un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_total.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento"')
plt.show()

In [ ]:
# Histograma de la distribución de los costos de mantenimiento
plt.figure(figsize=(16, 8))
sns.histplot(pow_total, kde=True)
plt.title('Distribución de los costos de mantenimiento. Normalizado')
plt.xlabel('Costo')
plt.ylabel('Frecuencia')
plt.show()

### Mantenimiento Correctivo

In [ ]:
# Frecuencias de mantenimiento correctivo por tipo de unidad (UnitType)
print(f'Frecuencia de mantenimiento correctivo por tipo de unidad:')
sayer_maint2_byUnit_corrective.groupby('UnitType')['RepairCount'].sum()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(sayer_maint2_byUnit_corrective['RepairCount'], kde=True)
plt.title('Distribución de las frecuencias de mantenimiento correctivo. RAW')
plt.xlabel('Frecuencia de mantenimiento correctivo')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(sayer_maint2_byUnit_corrective['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Añaadir la transformacion a una nueva columna
sayer_maint2_byUnit_corrective['RepairCount_norm'] = pow_repaircount

In [ ]:
# Creacion de un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_repaircount.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Frecuencia de mantenimiento correctivo"')
plt.show()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(pow_repaircount, kde=True)
plt.title('Distribución de las frecuencias de mantenimiento correctivo. Normalizado')
plt.xlabel('Frecuencia de mantenimiento correctivo.')
plt.ylabel('Cantidad de unidades')

### Mantenimiento Preventivo

In [ ]:
# Frecuencias de mantenimiento correctivo por tipo de unidad (UnitType)
print(f'Frecuencia de mantenimiento preventivo por tipo de unidad:')
sayer_maint2_byUnit_preventive.groupby('UnitType')['RepairCount'].sum()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(sayer_maint2_byUnit_preventive['RepairCount'], kde=True)
plt.title('Distribución de las frecuencias de mantenimiento preventivo. RAW')
plt.xlabel('Frecuencia de mantenimiento preventivo')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(sayer_maint2_byUnit_preventive['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Añadir la transformacion a una nueva columna
sayer_maint2_byUnit_preventive['RepairCount_norm'] = pow_repaircount

In [ ]:
# Creacion de un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_repaircount.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Frecuencia de mantenimiento preventivo"')
plt.show()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(pow_repaircount, kde=True)
plt.title('Distribución de las frecuencias de mantenimiento preventivo. Normalizado')
plt.xlabel('Frecuencia de mantenimiento preventivo.')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Eliminacion de las variables temporales utilizadas para la transformacion de las variables
del pow_horas, pow_total, pow_repaircount, scaler

# Hand's On 3A

## Correlaciones

### Preventive Freq. - Maint. Costs

In [ ]:
# Registros de data_sayer con ComplaintType 'PREVENTIVE'. Obtener el promedio de sus costos
sayer_maint2_byUnit_preventive['Total'] = data_sayer2[data_sayer2['ComplaintType'] == 'PREVENTIVO'].groupby('UnitID')['TOTAL'].sum().values

In [ ]:
# Sacar la correlacion entre las variables 'RepaitCount' y 'TotalCost' RAW
sayer_maint2_byUnit_preventive[['RepairCount', 'Total']].corr()

In [ ]:
# Grafica QQ-Plot para verificar la normalidad de los datos
stats.probplot(sayer_maint2_byUnit_preventive['Total'], dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento preventivo"')
plt.show()

In [ ]:
# Normalizar los datos de 'MeanCost'
scaler = PowerTransformer(method='yeo-johnson')
pow_meancost = scaler.fit_transform(sayer_maint2_byUnit_preventive['Total'].values.reshape(-1, 1))

In [ ]:
# Grafica QQ-Plot para verificar la normalidad de los datos
stats.probplot(pow_meancost.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento preventivo"')
plt.show()

In [ ]:
# Añadir la transformacion a una nueva columna
sayer_maint2_byUnit_preventive['Total_norm'] = pow_meancost

In [ ]:
# Sacar la correlacion entre las variables 'RepaitCount' y 'TotalCost' normalizadas
sayer_maint2_byUnit_preventive[['RepairCount_norm', 'Total_norm']].corr()

Como podemos observar, la relacion entre las columnas sin normalizar es mejor que las columnas normalizadas

In [ ]:
# Scatter plot between 'RepairCount' and 'Total'
plt.figure(figsize=(16, 8))
sns.scatterplot(x='RepairCount', y='Total', data=sayer_maint2_byUnit_preventive)
plt.title('Correlación entre Frecuencia y Costo de mantenimiento preventivo')
plt.xlabel('Frecuencia de mantenimiento preventivo')
plt.ylabel('Costo de mantenimiento preventivo')
plt.show()

#### Modelo de Regresión Lineal

In [ ]:
# Preparar el dataframe necesario para el modelo de regresion
# Debe incluir la frecuencia de Reparaciones por cada Unidad, Año Trimestre y Tipo de Reparacion
modelA = data_sayer2.groupby(['UnitID', 'MaintenanceYear', 'OpenedTrimester', 'ComplaintType'])['OpenedDate'].count().reset_index()

#Renombrar la columna 'OpenedDate' por 'RepairCount'
modelA.rename(columns={'OpenedDate': 'RepairCount'}, inplace=True)

# Añadir una columna 'Total' que contenga el costo total de las reparaciones
modelA['Total'] = data_sayer2.groupby(['UnitID', 'MaintenanceYear', 'OpenedTrimester', 'ComplaintType'])['TOTAL'].sum().values

# Obtener el dataframe para reparaciones preventivas
modelA_preventive = modelA[modelA['ComplaintType'] == 'PREVENTIVO'].copy()

In [ ]:
# Crear un modelo de regresion lineal, con las variables 'RepairCount' y 'Total'

X = modelA_preventive[['RepairCount']].values  # Variable independiente
y = modelA_preventive['Total'].values    # Variable dependiente

In [ ]:
# Dividir los datos en en train y test

# Datos de entrenamiento (Todos los de trimestres 1, 2 y 3 de 2022 y 2023)
X_train = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] != 4)][['RepairCount']].values
y_train = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] != 4)]['Total'].values

# Datos de prueba (Todos los de trimestre 4 de 2022 y 2023)
X_test = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] == 4)][['RepairCount']].values
y_test = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] == 4)]['Total'].values

# Creación del modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X=X_train, y=y_train)

In [ ]:
# Información del modelo
print(f"Intercept: {modelo.intercept_}")
print(f"Coeficiente: {modelo.coef_}")

In [ ]:
# Evaluación en el conjunto de entrenamiento
y_train_pred = modelo.predict(X_train)
print(f"R^2 en train: {modelo.score(X_train, y_train)}")

In [ ]:
# Evaluación en el conjunto de prueba
y_test_pred = modelo.predict(X_test)
print(f"R^2 en test: {r2_score(y_test, y_test_pred)}")
print(f"RMSE en test: {np.sqrt(mean_squared_error(y_test, y_test_pred))}")

In [ ]:
# Crear el modelo con statsmodels para obtener un resumen detallado
X_train_sm = sm.add_constant(X_train)  # Agregar término de intersección
modelo_sm = sm.OLS(y_train, X_train_sm).fit()
print(modelo_sm.summary())

In [ ]:
# Eliminacion de las variables temporales
del modelA, modelA_preventive
del X, X_train, X_train_sm, X_test, y, y_train, y_train_pred, y_test, y_test_pred,
del modelo, modelo_sm

### Corrective Freq. - UnitYear

In [ ]:
# Sacar la correlacion entre las variables 'RepairCount' y 'UnitYear'
sayer_maint2_byUnit_corrective[['RepairCount', 'UnitYear']].corr()

In [ ]:
# Normalizar los datos de 'RepairCount'
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(sayer_maint2_byUnit_corrective['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Añadir la transformacion a una nueva columna
sayer_maint2_byUnit_corrective['RepairCount_norm'] = pow_repaircount

In [ ]:
# Sacar la correlacion entre las variables 'RepairCount' y 'UnitYear' normalizadas
sayer_maint2_byUnit_corrective[['RepairCount_norm', 'UnitYear']].corr()

### MaintType - UnitYear

In [ ]:
# Crear una columna en data_sayer2 para saber si la reparacion fue correctiva o preventiva
data_sayer2['MaintType'] = data_sayer2['ComplaintType'].apply(lambda x: 1 if x == 'CORRECTIVO' else 0)

# Convertir a HTML

In [ ]:
# %%shell
# jupyter nbconvert --to html /content/E5_Construcción_y_transformación_de_datos.ipynb